<a href="https://colab.research.google.com/github/karanidenis/transfer_learning_assignment/blob/main/Transfer_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.losses import categorical_crossentropy
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras import layers, models
# from keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from skimage.transform import resize

In [3]:
IMG_ROWS = 224
IMG_COLS = 224
NUM_CLASSES = 10
TEST_SIZE = 0.2
RANDOM_STATE = 2018
NO_EPOCHS = 10
BATCH_SIZE = 128

learning_rate = 0.001

# Load pre-trained models
base_models = [VGG16, ResNet50, InceptionV3]
models = []

In [4]:
for base_model in base_models:
    base = base_model(weights='imagenet', include_top=False, input_shape=(IMG_ROWS, IMG_COLS, 3))
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1, activation='relu')(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=predictions)
    models.append(model)

In [5]:
# Freeze pre-trained layers
for model in models:
    for layer in model.layers[:-2]:
        layer.trainable = False

# Compile models
for model in models:
    model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    # model.summary()

In [6]:
# train_file = "fashion-mnist_train.csv"
train_file = "/content/drive/MyDrive/Colab Notebooks/fashion-mnist_train.csv"
# test_file  = "fashion-mnist_test.csv"
test_file = "/content/drive/MyDrive/Colab Notebooks/fashion-mnist_test.csv"

train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

print("Fashion MNIST train -  rows:",train_data.shape[0]," columns:", train_data.shape[1])
print("Fashion MNIST test -  rows:",test_data.shape[0]," columns:", test_data.shape[1])

Fashion MNIST train -  rows: 60000  columns: 785
Fashion MNIST test -  rows: 10000  columns: 785


In [7]:
# Create a dictionary for each type of label
labels = {0 : "T-shirt/top", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat",
          5: "Sandal", 6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle Boot"}

def get_classes_distribution(data):
    # Get the count for each label
    label_counts = data["label"].value_counts()

    # Get total number of samples
    total_samples = len(data)


    # Count the number of items in each class
    for i in range(len(label_counts)):
        label = labels[label_counts.index[i]]
        count = label_counts.values[i]
        percent = (count / total_samples) * 100
        print("{:<20s}:   {} or {}%".format(label, count, percent))

get_classes_distribution(train_data)


Pullover            :   6000 or 10.0%
Ankle Boot          :   6000 or 10.0%
Shirt               :   6000 or 10.0%
T-shirt/top         :   6000 or 10.0%
Dress               :   6000 or 10.0%
Coat                :   6000 or 10.0%
Sandal              :   6000 or 10.0%
Bag                 :   6000 or 10.0%
Sneaker             :   6000 or 10.0%
Trouser             :   6000 or 10.0%


In [8]:
def data_preprocessing(raw):
    # Convert labels to one-hot encoded vectors
    out_y = to_categorical(raw.label, NUM_CLASSES)
    num_images = raw.shape[0]
    x_as_array = raw.iloc[:, 1:].values.astype('float32')
    x_rgb_array = np.repeat(x_as_array.reshape(num_images, 28, 28, 1), 3, axis=-1)  # Resize to 3 channels
    x_resized = np.array([resize(img, (IMG_ROWS, IMG_COLS)) for img in x_rgb_array])  # Resize images
    out_x = x_resized / 255.0
    return out_x, out_y


# def data_preprocessing(raw):
#     out_y = to_categorical(raw.label, NUM_CLASSES)
#     num_images = raw.shape[0]
#     x_as_array = raw.values[:,1:]
#     # Convert grayscale images to RGB by duplicating the single channel
#     x_rgb_array = np.repeat(x_as_array.reshape(num_images, IMG_ROWS, IMG_COLS, 1), 3, axis=-1)
#     out_x = x_rgb_array / 255
#     return out_x, out_y


In [ ]:
X, y = data_preprocessing(train_data)
X_test, y_test = data_preprocessing(test_data)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [ ]:
print("Fashion MNIST train -  rows:",X_train.shape[0]," columns:", X_train.shape[1:4])
print("Fashion MNIST valid -  rows:",X_val.shape[0]," columns:", X_val.shape[1:4])
# print("Fashion MNIST test -  rows:",X_test.shape[0]," columns:", X_test.shape[1:4])
print(y_train.shape)

In [ ]:
train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

In [ ]:
import matplotlib.pyplot as plt

# Training the model
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=NO_EPOCHS,
                    verbose=1,
                    validation_data=(X_val, y_val))



# Evaluate the model on test data
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Plot training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()



In [ ]:
IMG_ROWS = 224
IMG_COLS = 224
NUM_CLASSES = 10
TEST_SIZE = 0.2
RANDOM_STATE = 2018
NO_EPOCHS = 10
BATCH_SIZE = 128

learning_rate = 0.001

# Load pre-trained models
base_models = [VGG16, ResNet50, InceptionV3]
models = []

for base_model in base_models:
    base = base_model(weights='imagenet', include_top=False, input_shape=(IMG_ROWS, IMG_COLS, 3))
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1, activation='relu')(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=predictions)
    models.append(model)

# Freeze pre-trained layers
for model in models:
    for layer in model.layers[:-2]:
        layer.trainable = False

# Compile models
for model in models:
    model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

# train_file = "fashion-mnist_train.csv"
train_file = "/content/drive/MyDrive/Colab Notebooks/fashion-mnist_train.csv"
# test_file  = "fashion-mnist_test.csv"
test_file = "/content/drive/MyDrive/Colab Notebooks/fashion-mnist_test.csv"

train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

print("Fashion MNIST train -  rows:",train_data.shape[0]," columns:", train_data.shape[1])
print("Fashion MNIST test -  rows:",test_data.shape[0]," columns:", test_data.shape[1])

def data_preprocessing(raw):
    # Convert labels to one-hot encoded vectors
    out_y = to_categorical(raw.label, NUM_CLASSES)

    # Reshape input images to the desired shape and normalize pixel values
    num_images = raw.shape[0]
    x_as_array = raw.iloc[:, 1:].values.astype('float32')  # Extract pixel values

    # Convert grayscale images to RGB by duplicating the single channel
    x_rgb_array = np.repeat(x_as_array.reshape(num_images, IMG_ROWS, IMG_COLS, 1), 3, axis=-1)

    out_x = x_rgb_array / 255.0

    return out_x, out_y

X, y = data_preprocessing(train_data)
X_test, y_test = data_preprocessing(test_data)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

print("Fashion MNIST train -  rows:",X_train.shape[0]," columns:", X_train.shape[1:4])
print("Fashion MNIST valid -  rows:",X_val.shape[0]," columns:", X_val.shape[1:4])
# print("Fashion MNIST test -  rows:",X_test.shape[0]," columns:", X_test.shape[1:4])
print(y_train.shape)

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))